# Environment

In [28]:
import gdown
import os
import lancedb
import re
import requests

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings

from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from typing_extensions import TypedDict
from typing import List
from langchain.schema import Document
from langgraph.graph import END, StateGraph

from ragas import SingleTurnSample,evaluate,EvaluationDataset
from ragas.metrics import faithfulness
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.vectorstores import LanceDB
from langchain.retrievers import BM25Retriever, EnsembleRetriever

from langchain.schema import Document
from langchain.retrievers import BM25Retriever

from langchain.text_splitter import CharacterTextSplitter
from bs4 import BeautifulSoup


from dotenv import load_dotenv
load_dotenv(".env")


True

## Vectorstore

In [29]:
# # 要抓取的網址列表
# urls = [
#     "https://forum.gamer.com.tw/C.php?bsn=79688&snA=154",
#     "https://forum.gamer.com.tw/C.php?bsn=79688&snA=65"
# ]

# # 儲存所有處理過的文本區塊
# documents = []

# # 依次處理每個網址
# for url in urls:
#     headers = {"User-Agent": "Mozilla/5.0"}
#     response = requests.get(url, headers=headers)

#     if response.status_code == 200:
#         html_content = response.text

#         # 使用 BeautifulSoup 提取純文本
#         soup = BeautifulSoup(html_content, "html.parser")
        
#         # 刪除留言區域
#         for comment_section in soup.find_all(class_="reply-content"):
#             comment_section.decompose()  # 刪除該區塊
        
#         # 提取純文本
#         text_content = soup.get_text(separator="\n")

#         # 清理文本
#         cleaned_text = re.sub(r'[ \u00a0]+', ' ', text_content)  # 替換多餘的空格
#         cleaned_text = re.sub(r'[✕]', '', cleaned_text)  # 移除不需要的特殊符號
#         cleaned_text = re.sub(r'(\d+\s*樓)|（刪除）.*?。|此文章已由原作者.*?刪除|LV\.\s*\d+|GP\s*\d+k?', '', cleaned_text)  # 刪除樓層標記和刪除提示
#         cleaned_text = re.sub(r'\d{4}-\d{2}-\d{2}\s*\d{2}:\d{2}:\d{2}', '', cleaned_text)  # 移除時間戳
#         cleaned_text = re.sub(r'(GP|BP)\s*-?\d+', '', cleaned_text)  # 移除 GP/BP 標記和數值
#         cleaned_text = re.sub(r'\n+', '\n', cleaned_text).strip()  # 移除多餘換行

#         # 初始化分割器
#         splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
#         sections = splitter.split_text(cleaned_text)

#         # 儲存每個網址的區塊資料
#         for idx, section in enumerate(sections, 1):
#             documents.append({
#                 "url": url,  # 儲存來源網址
#                 "section": f"# {idx}",
#                 "text": section
#             })

#         print(f"處理完成：{url}")
#     else:
#         print(f"無法訪問網址 {url}，狀態碼: {response.status_code}")

# # 顯示所有儲存的 documents
# print("所有 documents：")
# for doc in documents:
#     print(f"來源網址: {doc['url']}")
#     print(doc["section"])
#     print(doc["text"])
#     print("-" * 50)


In [30]:
# 要抓取的網址列表
urls = [
    "https://forum.gamer.com.tw/C.php?bsn=79688&snA=154",
    "https://forum.gamer.com.tw/C.php?bsn=79688&snA=65"
]

# 儲存所有處理過的文本區塊
documents = []

# 依次處理每個網址
for url in urls:
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        html_content = response.text

        # 使用 BeautifulSoup 提取純文本
        soup = BeautifulSoup(html_content, "html.parser")
        
        # 刪除留言區域
        for comment_section in soup.find_all(class_="reply-content"):
            comment_section.decompose()  # 刪除該區塊
        
        # 提取純文本
        text_content = soup.get_text(separator="\n")

        # 清理文本
        cleaned_text = re.sub(r'[ \u00a0]+', ' ', text_content)  # 替換多餘的空格
        cleaned_text = re.sub(r'[✕]', '', cleaned_text)  # 移除不需要的特殊符號
        cleaned_text = re.sub(r'(\d+\s*樓)|（刪除）.*?。|此文章已由原作者.*?刪除|LV\.\s*\d+|GP\s*\d+k?', '', cleaned_text)  # 刪除樓層標記和刪除提示
        cleaned_text = re.sub(r'\d{4}-\d{2}-\d{2}\s*\d{2}:\d{2}:\d{2}', '', cleaned_text)  # 移除時間戳
        cleaned_text = re.sub(r'(GP|BP)\s*-?\d+', '', cleaned_text)  # 移除 GP/BP 標記和數值
        cleaned_text = re.sub(r'\n+', '\n', cleaned_text).strip()  # 移除多餘換行
        cleaned_text = re.sub(r'^\s*0\s*回覆\s*', '', cleaned_text)  # 移除「0 回覆」
        cleaned_text = re.sub(r'\u200B', '', cleaned_text)  # 移除零寬度空格
        cleaned_text = re.sub(r'^\s*(還有|共)\s*\d+\s*(則留言|則回覆)\s*', '', cleaned_text)  # 移除留言數量標記
        cleaned_text = re.sub(r'(收合留言|顯示舊留言)', '', cleaned_text)  # 移除收合和顯示舊留言提示
        cleaned_text = re.sub(r'^\s*.*$', '', cleaned_text, flags=re.MULTILINE)  # 移除特殊字符和提示

        # 初始化分割器
        splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        sections = splitter.split_text(cleaned_text)

        # 儲存每個網址的區塊資料
        for idx, section in enumerate(sections, 1):
            documents.append({
                "url": url,  # 儲存來源網址
                "section": f"# {idx}",
                "text": section
            })

        print(f"處理完成：{url}")
    else:
        print(f"無法訪問網址 {url}，狀態碼: {response.status_code}")

# 顯示所有儲存的 documents
print("所有 documents：")
for doc in documents:
    print(f"來源網址: {doc['url']}")
    print(doc["section"])
    print(doc["text"])
    print("-" * 50)


Created a chunk of size 3836, which is longer than the specified 500
Created a chunk of size 1115, which is longer than the specified 500
Created a chunk of size 1702, which is longer than the specified 500
Created a chunk of size 678, which is longer than the specified 500
Created a chunk of size 538, which is longer than the specified 500
Created a chunk of size 659, which is longer than the specified 500
Created a chunk of size 887, which is longer than the specified 500
Created a chunk of size 1147, which is longer than the specified 500
Created a chunk of size 1105, which is longer than the specified 500
Created a chunk of size 516, which is longer than the specified 500
Created a chunk of size 1587, which is longer than the specified 500


處理完成：https://forum.gamer.com.tw/C.php?bsn=79688&snA=154


Created a chunk of size 5524, which is longer than the specified 500
Created a chunk of size 584, which is longer than the specified 500
Created a chunk of size 695, which is longer than the specified 500


處理完成：https://forum.gamer.com.tw/C.php?bsn=79688&snA=65
所有 documents：
來源網址: https://forum.gamer.com.tw/C.php?bsn=79688&snA=154
# 1
【心得】PTCG Pocket 各式牌組指南和分析 (各樓層更新國外比賽優勝牌組 #3,5,21更新其他牌型) @寶可夢集換式卡牌遊戲口袋版 哈啦板 - 巴哈姆特
找標題
查詢標題相符的文章
找作者
查詢作者相符的文章
找M文
查詢被 MARK 的文章
找全文
搜尋本板文章、作者
 
◄
►
1
2
【心得】PTCG Pocket 各式牌組指南和分析 (各樓層更新國外比賽優勝牌組 #3,5,21更新其他牌型)
綜合討論
樓主
MrWingdreams
善待新手・理性討論
MrWingdreams
BP
-
 編輯
前言:
由開服到現在 已經是每天都在玩了
每天就是等卡包 然後不停打對戰
這篇文章到正式上線同樣適用
希望可以幫助不少新入坑的朋友
目前42等 已完成所有圖鑑 歡迎加好友~
一起研究ptcgp的世界
除了必要卡外
其他卡片都是可以按自己喜歡作調整的
大家參考就好
 
每個人手型不一樣
1.皮卡丘ex
(必要)
皮卡丘x2 主力怪
閃電鳥x2 多數情況當肉盾 逆境也可拼運氣
小皮卡丘x2 進化用
雷丘x2 攻擊完會棄能量 尾刀用
博士的研究x2 抽濾必放2
精靈球x2 抽濾必放2
速度強化x2 這套打快攻 一定要2張
馬志士x2 配雷丘尾刀 1張有時候會抽不到
(非必要)
娜姿x2 因為是快攻有時候可以換出對手沒進化好的pkm打死 或是拖延對手
坂木x1 可以換其他 但坂木皮卡丘可以殺忍蛙,烈焰馬,阿栢怪等 手填的雷丘也可以殺超夢
而且各種傷藥 有時候差10點就是差10點
傷藥x1 這套牌血量都有點少
但有時候卡到20血就很賺 看個人
這一套就是一個字 快
理想情況為:
閃電鳥前排擋傷
後排雷丘+皮卡丘ex
皮卡丘ex充好就馬上輸出
雷丘最後尾刀
目前唯一差不多快的只有寶石海星
但寶石海星怕電 而且要1進 這套不用
這一套無疑是目前環境最強最快的牌組
(配雷彈/班馬也很強 不一定要玩雷丘)
缺點是前期養不好 中期開始就會無力
*我這配置只有6隻基礎+2球
太多基礎怪會拿不到皮卡丘ex 浪費前期優勢
太少

In [31]:
# 定義要使用的 Embedding 模型
embeddings = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-3-small",  
    openai_api_version="2023-05-15"            
)


1222

In [32]:
# 假設 documents 已經是分割後的文本資料
documents = []
for idx, section in enumerate(sections, 1):
    documents.append(Document(page_content=section, metadata={"section": f"# {idx}"}))

# Initialize the BM25 retriever
bm25_retriever = BM25Retriever.from_documents(documents)
bm25_retriever.k = 2  # Retrieve top 2 results

print("type of bm25", type(bm25_retriever))


type of bm25 <class 'langchain_community.retrievers.bm25.BM25Retriever'>


In [33]:
#建立 lancedb 向量儲存以進行密集語義搜尋/檢索
db = lancedb.connect("/tmp/lancedb")
table = db.create_table(
    "pandas_docs",
    data=[
        {
            "vector": embeddings.embed_query("Hello World"),
            "text": "Hello World",
            "id": "1",
        }
    ],
    mode="overwrite",
)


In [34]:
# 初始化 LanceDB 檢索器
docsearch = LanceDB.from_documents(documents, embeddings, connection=db)
retriever_lancedb = docsearch.as_retriever(search_kwargs={"k": 2})

# 初始化混合檢索器
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever_lancedb], weights=[0.4, 0.6]
)

1221

In [35]:
# 語意分割器: 定義 SemanticChunker
semantic_chunker = SemanticChunker(
    embeddings=embeddings,
    buffer_size=1,
    breakpoint_threshold_type='percentile',
    breakpoint_threshold_amount=0.9,
    min_chunk_size=100  # 最小的文本塊大小
)

# 使用 SemanticChunker 切分文本
docs_split = semantic_chunker.split_documents(documents)

## Web Search Tool

In [36]:
web_search_tool = TavilySearchResults()

## LLMs

### Question Router

In [37]:
# 定義兩個工具的 DataModel
class web_search(BaseModel):
    """
    網路搜尋工具。若問題與寶可夢ptcg"無關"，請使用web_search工具搜尋解答。
    """
    query: str = Field(description="使用網路搜尋時輸入的問題")

class vectorstore(BaseModel):
    """
    紀錄關於寶可夢ptcg的向量資料庫工具。若問題與寶可夢ptcg有關，請優先使用此工具搜尋解答。
    """
    query: str = Field(description="搜尋向量資料庫時輸入的問題")


# Prompt Template
instruction = """
你是將使用者問題導向向量資料庫或網路搜尋的專家。
向量資料庫包含有關寶可夢ptcg文件。對於這些主題的問題，請使用向量資料庫工具。如果找不到則使用網路搜尋工具。
"""
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",instruction),
        ("human", "{question}"),
    ]
)

# Route LLM with tools use
llm = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_CHAT_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
    temperature=0
)

structured_llm_router = llm.bind_tools(tools=[web_search, vectorstore])

# 使用 LCEL 語法建立 chain
question_router = route_prompt | structured_llm_router

In [38]:
# 測試 Route 功能
response = question_router.invoke({"question": "皮卡丘隊友那些必要卡片?"})
print(response.additional_kwargs['tool_calls'])
response = question_router.invoke({"question": "計算勞工的平均工資時，甚麼狀況不計入?"}) # 問題要明確
print(response.additional_kwargs['tool_calls'])
response = question_router.invoke({"question": "你好"}) # llm直接回答
print('tool_calls' in response.additional_kwargs)

[{'id': 'call_3Jrdqit5OKtRdyAm5XDEkgov', 'function': {'arguments': '{"query":"皮卡丘 必要卡片"}', 'name': 'vectorstore'}, 'type': 'function'}]
[{'id': 'call_900ONrqBy9PImBreTj3KgT9Z', 'function': {'arguments': '{"query":"計算勞工平均工資時不計入的狀況"}', 'name': 'web_search'}, 'type': 'function'}]
False


### RAG Responder

In [39]:
# Prompt Template
instruction = """
你是一位負責處理使用者問題的助手，請利用提取出來的文件內容來回應問題。
若問題的答案無法從文件內取得，請直接回覆你不知道，禁止虛構答案。
注意：請確保答案的準確性。
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",instruction),
        ("system","文件: \n\n {documents}"),
        ("human","問題: {question}"),
    ]
)

# LLM & chain
rag_chain = prompt | llm | StrOutputParser()

In [40]:
# 測試 rag_chain 功能
question = "皮卡丘隊友那些必要卡片?"
docs = ensemble_retriever.invoke(question)

generation = rag_chain.invoke({"documents": docs, "question": question})
print(generation)

文件中並未提供有關皮卡丘隊友必要卡片的具體資訊，因此我不知道這個問題的答案。


### Plain LLM

In [41]:
# Prompt Teamplate
instruction = """
你是一位負責處理使用者問題的助手，請利用你的知識來回應問題。
回應問題時請確保答案的準確性，勿虛構答案。
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",instruction),
        ("human","問題: {question}"),
    ]
)

# LLM & chain
llm_chain = prompt | llm | StrOutputParser()

In [42]:
# 測試 llm_chain 功能
question = "Hi how are you?"
generation = llm_chain.invoke({"question": question})
print(generation)

Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?


### Retrieval Grader

In [43]:
class GradeDocuments(BaseModel):
    """
    確認提取文章與問題是否有關(yes/no)
    """

    binary_score: str = Field(description="請問文章與問題是否相關。('yes' or 'no')")

# Prompt Template
instruction = """
你是一個評分的人員，負責評估文件與使用者問題的關聯性。
如果文件包含與使用者問題相關的關鍵字或語意，則將其評為相關。
輸出 'yes' or 'no' 代表文件與問題的相關與否。
"""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",instruction),
        ("human", "文件: \n\n {document} \n\n 使用者問題: {question}"),
    ]
)

# Grader LLM
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# 使用 LCEL 語法建立 chain
retrieval_grader = grade_prompt | structured_llm_grader

In [44]:
# 測試 grader 功能
question = "一張超夢建議可以怎麼組隊??"
docs = ensemble_retriever.invoke(question)

doc_txt = docs[0].page_content
response =  retrieval_grader.invoke({"question": question, "document": doc_txt})
print(response)

binary_score='yes'


In [45]:
docs

[Document(metadata={'page': None, 'source': None}, page_content='JexSlam\nJexSlam\nBP\n-\n延續組牌的概念，我也組了一套超夢牌組，\n目前打了一週下來，勝率大約有8成，運作起來還算順暢。\n我組牌的核心概念，就是運用遊戲機制，\n「起手一定有一張基礎寶可夢」\n也因此我的超夢牌組，\n只有超夢EX，以及沙奈朵家族。\n>> 也就是我的\n起手一定會有一張超夢或拉魯拉絲\n我這套牌組，有個地方可以依照個人習慣調整：\n>> 板木老大：我放了兩張，是因為超夢的2能攻擊力只有50，但是靠著板木老大，\n60的攻擊力可以打掉很多基礎寶可夢\n，甚至是在最後\n尾刀時可以發揮奇效\n（很多人算牌的時候會漏算這10點攻擊力XD）\n>> 紅牌：板木老大有個問題是，他是支援者，所以有時候會卡到其他兩張，所以也可以考慮放紅牌。現在大部分的牌組都仰賴不斷的進化和抽牌濾牌，所以有時候手上扣一張紅牌，也是不錯的選擇。（基本上\n對手手牌超過3張，就可以丟紅牌了\n，就算沒丟到進化寶可夢，讓對手少一張手牌也是賺）\n牌組打法：\n基本上這套牌組的\n核心是沙奈朵\n，如果是內戰，誰先進化出沙奈朵誰就贏了。\n>> 請留意\n沙奈朵的技能是戰鬥區充能\n，有時候把能量一個放備戰區、一個放戰鬥區，也可以慢慢養出第二隻超夢。\n我通常會配合板木老大，用50+10的攻擊力，從前期就想辦法壓制對手的基礎寶可夢。\n>> 透過娜姿，可以把對方想養的寶可夢抓上來打，超夢2點打50的速度其實算蠻快的。\n玩超夢建議不要拖到大後期。\n>> 比如噴火龍或者是其他超過150滴血的寶可夢，會是很麻煩的存在，也因此我們的\n目標就是在中期接近後期時，趕快把對手擊敗\n，才不會遇到血太厚的寶可夢把自已磨死。\n3\n-\n回覆'),
 Document(metadata={'page': None, 'source': None}, page_content='【心得】PTCGP新手問題大合輯(牌組分享、活動牌組推薦、單人精英任務、卡包建議) - 2024/12/16更新 @寶可夢集換式卡牌遊戲口袋版 哈啦板 - 巴哈姆特\n找標題\n查詢標題相符的文章\n找作者\n查詢作者相符的文章\n找M文\n查詢被 MARK 的文章\n找

### Answer Grader

In [46]:
class GradeAnswer(BaseModel):
    """
    確認答案是否可回應問題
    """

    binary_score: str = Field(description="答案是否回應問題。('yes' or 'no')")

# Prompt Template
instruction = """
你是一個評分的人員，負責確認答案是否回應了問題。
輸出 'yes' or 'no'。 'Yes' 代表答案確實回應了問題， 'No' 則代表答案並未回應問題。
"""
# Prompt
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",instruction),
        ("human", "使用者問題: \n\n {question} \n\n 答案: {generation}"),
    ]
)

# LLM with function call
structured_llm_grader = llm.with_structured_output(GradeAnswer)

# 使用 LCEL 語法建立 chain
answer_grader = answer_prompt | structured_llm_grader

In [47]:
#測試 grader 功能
question = "皮卡丘基本隊伍配置"
docs = ensemble_retriever.invoke(question)

generation = rag_chain.invoke({"documents": docs, "question": question})
answer_grader.invoke({"question": question,"generation": generation})

GradeAnswer(binary_score='no')

# Graph

## Graph state

In [48]:

class GraphState(TypedDict):
    """
    State of graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
    """
    question : str
    generation : str
    documents : List[str]

## Nodes / Conditional edges

In [49]:
def retrieve(state):
    """
    Retrieve documents related to the question.

    Args:
        state (dict):  The current state graph

    Returns:
        state (dict): New key added to state, documents, that contains list of related documents.
    """

    print("---RETRIEVE---")
    question = state["question"]

    #Retrieval 
    documents =  ensemble_retriever.invoke(question)

    
    print(documents)
    return {"documents":documents, "question":question}

def web_search(state):
    """
    Web search based on the re-phrased question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates documents key with appended web results
    """

    print("---WEB SEARCH---")
    question = state["question"]
    
    documents = state["documents"] if state["documents"] else []
    # print(documents)
    # state.get(retriever)

    
    # Web search
    docs = web_search_tool.invoke({"query": question})
    web_results = [Document(page_content=d["content"]) for d in docs]
    
    # print(web_results)
    # print("web_search",web_results)

    # web_results含有documents
    documents = documents + web_results
    print(documents)
    return {"documents": documents, "question": question}

def retrieval_grade(state):
    """
    filter retrieved documents based on question.

    Args:
        state (dict):  The current state graph

    Returns:
        state (dict): New key added to state, documents, that contains list of related documents.
    """

    # Grade documents
    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")

    documents = state["documents"]
    question = state["question"]

    # Score each doc
    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke({"question": question, "document": d.page_content})
        grade = score.binary_score
        if grade == "yes":
            print("  -GRADE: DOCUMENT RELEVANT-")
            filtered_docs.append(d)
        else:
            print("  -GRADE: DOCUMENT NOT RELEVANT-")
            continue
    return {"documents": filtered_docs, "question": question}

def rag_generate(state):
    """
    Generate answer using  vectorstore / web search

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """

    print("---GENERATE IN RAG MODE---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain.invoke({"documents": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}

def plain_answer(state):
    """
    Generate answer using the LLM without vectorstore.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """

    print("---GENERATE PLAIN ANSWER---")
    question = state["question"]
    generation = llm_chain.invoke({"question": question})
    return {"question": question, "generation": generation}


### Edges ###
def route_question(state):
    """
    Route question to web search or RAG.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    print("---ROUTE QUESTION---")
    question = state["question"]
    source = question_router.invoke({"question": question})

    # Fallback to plain LLM or raise error if no decision
    if "tool_calls" not in source.additional_kwargs:
        print("  -ROUTE TO PLAIN LLM-")
        return "plain_answer"
    if len(source.additional_kwargs["tool_calls"]) == 0:
      raise "Router could not decide source"

    # Choose datasource
    datasource = source.additional_kwargs["tool_calls"][0]["function"]["name"]
    if datasource == 'web_search':
        print("  -ROUTE TO WEB SEARCH-")
        return "web_search"
    elif datasource == 'vectorstore':
        print("  -ROUTETO VECTORSTORE-")
        return "vectorstore"

def route_retrieval(state):
    """
    Determines whether to generate an answer, or use websearch.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ROUTE RETRIEVAL---")
    filtered_documents = state["documents"]

    if not filtered_documents:
        # All documents have been filtered check_relevance
        print("  -DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, ROUTE TO WEB SEARCH-")
        return "web_search"
    else:
        # We have relevant documents, so generate answer
        print("  -DECISION: GENERATE WITH RAG LLM-")
        return "rag_generate"


In [50]:
def grade_rag_generation(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]
    
    print("State content:", type(state),state)
    contexts = [doc.page_content for doc in documents]  # Extract page content from documents
    sample = SingleTurnSample(
        user_input=question,
        response=generation,
        retrieved_contexts=contexts,
    )
    dataset = EvaluationDataset([sample])
    print("State content:", type(dataset))
    print(dataset)
    faithfulness_score = evaluate(dataset=dataset,metrics=[faithfulness],llm=llm)
    print(type(faithfulness_score),faithfulness_score)
    score = float(faithfulness_score['faithfulness'][0])
    print(type(score),score)
    if score < 0.5:
        print("  -DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY-")
        return "not supported"

    else:
        print("  -DECISION: GENERATION IS GROUNDED IN DOCUMENTS-")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question,"generation": generation})
        grade = score.binary_score
        if grade == "yes":
            print("  -DECISION: GENERATION ADDRESSES QUESTION-")
            return "useful"
        else:
            print("  -DECISION: GENERATION DOES NOT ADDRESS QUESTION-")
            return "not useful"

## Build Graph

In [51]:
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve", retrieve) # retrieve
workflow.add_node("web_search", web_search) # web search
workflow.add_node("retrieval_grade", retrieval_grade) # retrieval grade
workflow.add_node("rag_generate", rag_generate) # rag
workflow.add_node("plain_answer", plain_answer) # llm

# Build graph
workflow.set_conditional_entry_point(
    route_question,
    {
        "vectorstore": "retrieve",
        "web_search": "web_search",
        "plain_answer": "plain_answer",
    },
)
workflow.add_edge("retrieve", "retrieval_grade")
workflow.add_edge("web_search", "retrieval_grade")
workflow.add_conditional_edges(
    "retrieval_grade",
    route_retrieval,
    {
        "rag_generate": "rag_generate",
        "web_search": "web_search",
    },
)
workflow.add_conditional_edges(
    "rag_generate",
    grade_rag_generation,
    {
        "not supported": "rag_generate", # Hallucinations: re-generate
        "not useful": "web_search", # Fails to answer question: fall-back to web-search
        "useful": END,
    },
)
workflow.add_edge("plain_answer", END)

# Compile
app = workflow.compile()

### 實際測試

In [52]:
def run(question):
    inputs = {"question": question,"documents":None}
    for output in app.stream(inputs):
        print("\n")

    # Final 
    if 'rag_generate' in output.keys():
        print(output['rag_generate']['generation'])
    elif 'plain_answer' in output.keys():
        print(output['plain_answer']['generation'])

In [53]:
run("現今中國統治者是誰?")

---ROUTE QUESTION---
  -ROUTE TO WEB SEARCH-
---WEB SEARCH---
[Document(metadata={}, page_content='在中國兩千多年的漫長帝制社會裡，皇帝是國家的最高統治者，是專制統治的象徵與代表。 據統計，自公元前221年秦王嬴政稱皇帝始，到1912年「末代皇帝」溥儀退位，在不足2132年中，中國共有皇帝422人。'), Document(metadata={}, page_content='編輯 中國共產黨 歷任中華人民共和國國家元首列表[編輯] 中國共產黨 \xa0\xa0多黨聯合代行職權 （1893—1976） 1   1949年10月1日  1954年9月27日  1949 — 99.83%   \xa0中國共產黨 （政務院總理） 自己 （1893—1976） 1   1954年9月27日  1959年4月27日  1954 — 100.00%  \xa0中國共產黨 （中央主席）  朱德  周恩來 自己 （1898—1969） 2   1959年4月27日  1964年12月21日 1959 — 98.94%   \xa0中國共產黨 （中央副主席） 宋慶齡、董必武 周恩來 毛澤東 （1886—1975） -   1972年2月24日  1975年1月17日  接任代理主席  \xa0中國共產黨 C1      朱德 （本次選舉為委員長選舉）    \xa0中國共產黨 （中央政治局常委）   不適用 周恩來、華國鋒 毛澤東 C2      宋慶齡 C3      葉劍英 （本次選舉為委員長選舉）    \xa0中國共產黨 （中央政治局常委）   不適用 華國鋒、趙紫陽 鄧小平 （1909—1992） 4   1983年6月18日  1988年4月8日   1983 — 無數據  \xa0中國共產黨 （1907—1998） 5   1988年4月8日   1993年3月27日  1988 — 94.68%   \xa0中國共產黨 （1926—2022） 6   1993年3月27日  1998年3月5日   1993 — 97.94%[註 1]  \xa0中國共產黨 （1942—） 8   2003年3月15日  2008年3月5日   2003 — 99.69%   \xa0中國

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

<class 'ragas.dataset_schema.EvaluationResult'> {'faithfulness': 0.0000}
<class 'float'> 0.0
  -DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY-


---GENERATE IN RAG MODE---
---CHECK HALLUCINATIONS---
State content: <class 'dict'> {'documents': [Document(metadata={}, page_content='編輯 中國共產黨 歷任中華人民共和國國家元首列表[編輯] 中國共產黨 \xa0\xa0多黨聯合代行職權 （1893—1976） 1   1949年10月1日  1954年9月27日  1949 — 99.83%   \xa0中國共產黨 （政務院總理） 自己 （1893—1976） 1   1954年9月27日  1959年4月27日  1954 — 100.00%  \xa0中國共產黨 （中央主席）  朱德  周恩來 自己 （1898—1969） 2   1959年4月27日  1964年12月21日 1959 — 98.94%   \xa0中國共產黨 （中央副主席） 宋慶齡、董必武 周恩來 毛澤東 （1886—1975） -   1972年2月24日  1975年1月17日  接任代理主席  \xa0中國共產黨 C1      朱德 （本次選舉為委員長選舉）    \xa0中國共產黨 （中央政治局常委）   不適用 周恩來、華國鋒 毛澤東 C2      宋慶齡 C3      葉劍英 （本次選舉為委員長選舉）    \xa0中國共產黨 （中央政治局常委）   不適用 華國鋒、趙紫陽 鄧小平 （1909—1992） 4   1983年6月18日  1988年4月8日   1983 — 無數據  \xa0中國共產黨 （1907—1998） 5   1988年4月8日   1993年3月27日  1988 — 94.68%   \xa0中國共產黨 （1926—2022） 6   1993年3月27日  1998年3月5日   1993 — 97.94%[註 1]  \xa0中國共產黨 （1942

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

<class 'ragas.dataset_schema.EvaluationResult'> {'faithfulness': 0.0000}
<class 'float'> 0.0
  -DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY-


---GENERATE IN RAG MODE---
---CHECK HALLUCINATIONS---
State content: <class 'dict'> {'documents': [Document(metadata={}, page_content='編輯 中國共產黨 歷任中華人民共和國國家元首列表[編輯] 中國共產黨 \xa0\xa0多黨聯合代行職權 （1893—1976） 1   1949年10月1日  1954年9月27日  1949 — 99.83%   \xa0中國共產黨 （政務院總理） 自己 （1893—1976） 1   1954年9月27日  1959年4月27日  1954 — 100.00%  \xa0中國共產黨 （中央主席）  朱德  周恩來 自己 （1898—1969） 2   1959年4月27日  1964年12月21日 1959 — 98.94%   \xa0中國共產黨 （中央副主席） 宋慶齡、董必武 周恩來 毛澤東 （1886—1975） -   1972年2月24日  1975年1月17日  接任代理主席  \xa0中國共產黨 C1      朱德 （本次選舉為委員長選舉）    \xa0中國共產黨 （中央政治局常委）   不適用 周恩來、華國鋒 毛澤東 C2      宋慶齡 C3      葉劍英 （本次選舉為委員長選舉）    \xa0中國共產黨 （中央政治局常委）   不適用 華國鋒、趙紫陽 鄧小平 （1909—1992） 4   1983年6月18日  1988年4月8日   1983 — 無數據  \xa0中國共產黨 （1907—1998） 5   1988年4月8日   1993年3月27日  1988 — 94.68%   \xa0中國共產黨 （1926—2022） 6   1993年3月27日  1998年3月5日   1993 — 97.94%[註 1]  \xa0中國共產黨 （1942

...

<class 'ragas.dataset_schema.EvaluationResult'> {'faithfulness': 0.0000}
<class 'float'> 0.0
  -DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY-


---GENERATE IN RAG MODE---
---CHECK HALLUCINATIONS---
State content: <class 'dict'> {'documents': [Document(metadata={}, page_content='編輯 中國共產黨 歷任中華人民共和國國家元首列表[編輯] 中國共產黨 \xa0\xa0多黨聯合代行職權 （1893—1976） 1   1949年10月1日  1954年9月27日  1949 — 99.83%   \xa0中國共產黨 （政務院總理） 自己 （1893—1976） 1   1954年9月27日  1959年4月27日  1954 — 100.00%  \xa0中國共產黨 （中央主席）  朱德  周恩來 自己 （1898—1969） 2   1959年4月27日  1964年12月21日 1959 — 98.94%   \xa0中國共產黨 （中央副主席） 宋慶齡、董必武 周恩來 毛澤東 （1886—1975） -   1972年2月24日  1975年1月17日  接任代理主席  \xa0中國共產黨 C1      朱德 （本次選舉為委員長選舉）    \xa0中國共產黨 （中央政治局常委）   不適用 周恩來、華國鋒 毛澤東 C2      宋慶齡 C3      葉劍英 （本次選舉為委員長選舉）    \xa0中國共產黨 （中央政治局常委）   不適用 華國鋒、趙紫陽 鄧小平 （1909—1992） 4   1983年6月18日  1988年4月8日   1983 — 無數據  \xa0中國共產黨 （1907—1998） 5   1988年4月8日   1993年3月27日  1988 — 94.68%   \xa0中國共產黨 （1926—2022） 6   1993年3月27日  1998年3月5日   1993 — 97.94%[註 1]  \xa0中國共產黨 （1942

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

<class 'ragas.dataset_schema.EvaluationResult'> {'faithfulness': 0.0000}
<class 'float'> 0.0
  -DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY-


---GENERATE IN RAG MODE---
---CHECK HALLUCINATIONS---
State content: <class 'dict'> {'documents': [Document(metadata={}, page_content='編輯 中國共產黨 歷任中華人民共和國國家元首列表[編輯] 中國共產黨 \xa0\xa0多黨聯合代行職權 （1893—1976） 1   1949年10月1日  1954年9月27日  1949 — 99.83%   \xa0中國共產黨 （政務院總理） 自己 （1893—1976） 1   1954年9月27日  1959年4月27日  1954 — 100.00%  \xa0中國共產黨 （中央主席）  朱德  周恩來 自己 （1898—1969） 2   1959年4月27日  1964年12月21日 1959 — 98.94%   \xa0中國共產黨 （中央副主席） 宋慶齡、董必武 周恩來 毛澤東 （1886—1975） -   1972年2月24日  1975年1月17日  接任代理主席  \xa0中國共產黨 C1      朱德 （本次選舉為委員長選舉）    \xa0中國共產黨 （中央政治局常委）   不適用 周恩來、華國鋒 毛澤東 C2      宋慶齡 C3      葉劍英 （本次選舉為委員長選舉）    \xa0中國共產黨 （中央政治局常委）   不適用 華國鋒、趙紫陽 鄧小平 （1909—1992） 4   1983年6月18日  1988年4月8日   1983 — 無數據  \xa0中國共產黨 （1907—1998） 5   1988年4月8日   1993年3月27日  1988 — 94.68%   \xa0中國共產黨 （1926—2022） 6   1993年3月27日  1998年3月5日   1993 — 97.94%[註 1]  \xa0中國共產黨 （1942

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

<class 'ragas.dataset_schema.EvaluationResult'> {'faithfulness': 1.0000}
<class 'float'> 1.0
  -DECISION: GENERATION IS GROUNDED IN DOCUMENTS-
---GRADE GENERATION vs QUESTION---
  -DECISION: GENERATION ADDRESSES QUESTION-


現今中國的統治者是習近平。


In [54]:
run("苗栗是不是國家")

---ROUTE QUESTION---
  -ROUTE TO WEB SEARCH-
---WEB SEARCH---
[Document(metadata={}, page_content="微紀事｜用中共邏輯論證，對於一個中國人而言，苗栗是一個獨立的國家 - 野人 (@Yeren) Enter 微紀事｜用中共邏輯論證，對於一個中國人而言，苗栗是一個獨立的國家 野人 Aug 27 IPFS Translate 中共邏輯真神奇！ 對於中國而言，台灣本身的國家屬性是曖昧不明的，其在政治上輸出的理由大多是： 1、台灣跟中國原本屬於同一個政治體； 2、台灣只是一個地理概念，而非政治概念； 3、台灣文化的根源在中國； 4、國際社會認同世界上只有一個中國。 那麼，根據中國人認為台灣並非獨立國家的政治理由，可得： 1、苗栗跟中國擁有兩種完全不同的政治制度，苗栗也並未奉行中國的特殊政治制度（一國兩制），所以兩邊在事實上並不屬於同一政治團體； 2、苗栗並不只是一個地理概念，其獨特的文化以及與周邊城市明確的政治性邊界劃分，使「苗栗」本身成為一個區別於周邊所有鄰近政治體的獨立政治概念； 3、苗栗的文化根源國並不是中國，因為其受到外來文化影響的階段（清代，受滿清時期漢文化影響），彼時「中國」一詞本身還未成為指代清政府的政治概念； 4、國際社會認同只有一個苗栗，假使中國不承認自己隸屬于苗栗縣，那麼苗栗就是一個獨立的國家。 所以，對於一個認為台灣不是一個獨立國家的中國人而言，苗栗是一個獨立的國家。 沒有用的後記 是不是傳言這麼久了，還沒有人仔細(?)論證過苗栗是一個獨立的國家~_~ 所以說喔，下次去苗栗記得帶護照。 半夜看完書睡不著起來寫廢文~_~ 最近大腦當機~_~ 導致~_~ 顏文字都在用同一個~_~ 沒救了~_~ 其實一直有想過聊神奇的中共邏輯，結合了劣化版的馬克思主義跟傳統中式邏輯的東西，神奇的是普及率高且深入人心，即便有人因為受不了中共的限制逃到國外，口尊「自由民主」，但思維方式還是原本的，只是立場換掉了，就想說獨裁式思維放到民主語境也適用，民主本身還滿有潛力嘛（笑死） 話說馬克思主義給我的印象沒有很「學術」，有點意識形態的傾向，忘記之前有一門什麼課講到馬克思，好像是現代哲學之類的？（反正不是政治哲學，因為那門課我全部翹掉最後18分混過~_~）然後教授講到馬克思那一章就滿臉嫌棄

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

<class 'ragas.dataset_schema.EvaluationResult'> {'faithfulness': 1.0000}
<class 'float'> 1.0
  -DECISION: GENERATION IS GROUNDED IN DOCUMENTS-
---GRADE GENERATION vs QUESTION---
  -DECISION: GENERATION ADDRESSES QUESTION-


根據文件內容，苗栗被戲稱為「苗栗國」，並且有居民堅持苗栗自成一國的觀點。然而，這些觀點並不代表苗栗在國際法上被承認為一個獨立國家。因此，苗栗並不是一個被正式承認的國家。


In [55]:
run("超夢隊要怎麼玩")

---ROUTE QUESTION---
  -ROUTETO VECTORSTORE-
---RETRIEVE---
[Document(metadata={'section': '# 7'}, page_content='立哥\nandychung00\nBP\n-\n 編輯\n\ue32c\n請問物品卡跟訓練家卡也是卡包抽得到嗎 目前傷藥跟速度強化都只有新手任務送的一個\n1\n-\n回覆\n\n擦低\nzxcv40711\nBP\n-\n\ue32c\n請問單抽跟十抽有差嗎\n1\n-\n回覆\n\n\njake\njake8220\nBP\n-\n\ue32c\n6878888472466097\n好友++會長期玩\n1\n-\n回覆\n\nUen10062\nzx963302\nBP\n-\n\ue32c\n這遊戲是設定沒有場地魔法卡嗎？還是單純早期實體卡沒有場地卡？\n1\n-\n回覆\n\n\n浩\njefferyxx\nBP\n-\n作者標示-非商業性\n本授權條款允許使用者重製、散布、傳輸以及修改著作，但不得為商業目的之使用。使用時必須按照著作人指定的方式表彰其姓名。\n問一下對戰時把對面寶可夢打死,為何有時候是+1分 有時候是+2分?\n3\n-\n回覆\n\n迴夢淨塵\ndark92110\nBP\n-\n爬文知道大概跟遊戲王DL一樣是簡化規則\n但是這個是初代嗎? 如果以收集的角度來看還不錯\n可是以PVP來說的話卡池有點少，戰術性也沒那麼多選擇\n之後出的卡會改變這種情況嗎?\n1\n-\n回覆'), Document(metadata={'page': None, 'source': None}, page_content='JexSlam\nJexSlam\nBP\n-\n延續組牌的概念，我也組了一套超夢牌組，\n目前打了一週下來，勝率大約有8成，運作起來還算順暢。\n我組牌的核心概念，就是運用遊戲機制，\n「起手一定有一張基礎寶可夢」\n也因此我的超夢牌組，\n只有超夢EX，以及沙奈朵家族。\n>> 也就是我的\n起手一定會有一張超夢或拉魯拉絲\n我這套牌組，有個地方可以依照個人習慣調整：\n>> 板木老大：我放了兩張，是因為超夢的2能攻擊力只有50，但是靠著板木老大，\n60的攻擊力可以打掉很多基礎寶可夢\n，甚至是在最後\n尾刀

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

<class 'ragas.dataset_schema.EvaluationResult'> {'faithfulness': 1.0000}
<class 'float'> 1.0
  -DECISION: GENERATION IS GROUNDED IN DOCUMENTS-
---GRADE GENERATION vs QUESTION---
  -DECISION: GENERATION ADDRESSES QUESTION-


超夢隊的玩法核心在於利用沙奈朵的技能和板木老大的支援來壓制對手的基礎寶可夢。以下是一些關鍵策略：

1. **起手寶可夢**：確保起手有一張基礎寶可夢，通常是超夢或拉魯拉絲。

2. **利用沙奈朵**：沙奈朵的技能「戰鬥區充能」可以幫助你在戰鬥區和備戰區之間分配能量，這樣可以慢慢養出第二隻超夢。

3. **攻擊策略**：使用板木老大來提升超夢的攻擊力，從而在前期壓制對手的基礎寶可夢。板木老大可以讓超夢的攻擊力從50提升到60，這對於擊敗許多基礎寶可夢非常有效。

4. **控制對手手牌**：可以考慮使用紅牌來減少對手的手牌數量，這樣即使沒有丟到進化寶可夢，也能讓對手少一張手牌。

5. **中期壓制**：在中期接近後期時，盡快擊敗對手，避免對方出場血量過高的寶可夢（如噴火龍），這樣可以減少被磨死的風險。

總之，超夢隊的關鍵在於快速進化和有效的能量管理，並在適當的時機進行攻擊。


In [56]:
run("在ptcg裡面玩甚麼隊伍勝率最高")

---ROUTE QUESTION---
  -ROUTETO VECTORSTORE-
---RETRIEVE---
[Document(metadata={'section': '# 8'}, page_content='連結\n文字顏色\n更多顏色\n背景顏色\n更多顏色\n中\n特大\n很大\n大\n中\n小\n字級\n粗體\n斜體\n底線\n刪除線\n清除格式\n更多顏色\n未登入的勇者，要加入 的討論嗎？\n延伸閱讀\n討論\n非主流牌組分享 小剛大岩蛇忍蛙牌組(無ex)\n心得\n蛋花牌組分享，疑？這朵花怎麼是大食花！？\n攻略\n我玩過最有趣的牌組 聒噪鳥&暴鯉\n心得\n重發重編輯 蜈蚣噴火龍牌組分享\n心得\n牌組分享-睡睡忍蛙牌組\n心得\n幻遊島環境-首週"賽場"環境\n情報\n【Pokemon TCG】【未來活動及卡牌大爆\n心得\n聒噪鳥暴鯉龍的最終型態 - 還得是靠\n◄\n►\n1\n2\n3\n4\n...\n7\n板務人員：\nIP 位置\n發文時間\n編輯留言\n刪除留言\n檢舉留言\n作品介紹\n水桶名單\n分享\n關閉圖片影片\n設定\n回覆文章\n卡回文\n\ue314\n【心得】PTCGP新手問題大合輯(牌組分享、活動牌組推薦、單人精英任務、卡包建議) - 2024/12/16更新\n一樓\n最新\n本站所刊載之圖文內容等版權皆屬原廠商或原作者所有，非經同意請勿轉載\n巴 哈 姆 特 電 玩 資 訊 站 https://www.gamer.com.tw'), Document(metadata={'page': None, 'source': None}, page_content='【心得】PTCGP新手問題大合輯(牌組分享、活動牌組推薦、單人精英任務、卡包建議) - 2024/12/16更新 @寶可夢集換式卡牌遊戲口袋版 哈啦板 - 巴哈姆特\n找標題\n查詢標題相符的文章\n找作者\n查詢作者相符的文章\n找M文\n查詢被 MARK 的文章\n找全文\n搜尋本板文章、作者\n \n◄\n►\n1\n2\n3\n4\n...\n7\n【心得】PTCGP新手問題大合輯(牌組分享、活動牌組推薦、單人精英任務、卡包建議) - 2024/12/16更新\n綜合討論\n樓主\nJexSlam\n立志成為優文大師！\nJexSl

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

<class 'ragas.dataset_schema.EvaluationResult'> {'faithfulness': 0.5000}
<class 'float'> 0.5
  -DECISION: GENERATION IS GROUNDED IN DOCUMENTS-
---GRADE GENERATION vs QUESTION---
  -DECISION: GENERATION ADDRESSES QUESTION-


根據文件內容，牌組強度排名中提到的勝率較高的牌組為超夢和皮卡丘，並且提到超夢的勝率約有8成。其他提到的牌組如噴火龍則被認為是強度較高的選擇，但具體的勝率並未明確列出。


In [57]:
run("在ptcg裡面那些隊伍可以最快靠賽獲勝")

---ROUTE QUESTION---
  -ROUTETO VECTORSTORE-
---RETRIEVE---
[Document(metadata={'section': '# 8'}, page_content='連結\n文字顏色\n更多顏色\n背景顏色\n更多顏色\n中\n特大\n很大\n大\n中\n小\n字級\n粗體\n斜體\n底線\n刪除線\n清除格式\n更多顏色\n未登入的勇者，要加入 的討論嗎？\n延伸閱讀\n討論\n非主流牌組分享 小剛大岩蛇忍蛙牌組(無ex)\n心得\n蛋花牌組分享，疑？這朵花怎麼是大食花！？\n攻略\n我玩過最有趣的牌組 聒噪鳥&暴鯉\n心得\n重發重編輯 蜈蚣噴火龍牌組分享\n心得\n牌組分享-睡睡忍蛙牌組\n心得\n幻遊島環境-首週"賽場"環境\n情報\n【Pokemon TCG】【未來活動及卡牌大爆\n心得\n聒噪鳥暴鯉龍的最終型態 - 還得是靠\n◄\n►\n1\n2\n3\n4\n...\n7\n板務人員：\nIP 位置\n發文時間\n編輯留言\n刪除留言\n檢舉留言\n作品介紹\n水桶名單\n分享\n關閉圖片影片\n設定\n回覆文章\n卡回文\n\ue314\n【心得】PTCGP新手問題大合輯(牌組分享、活動牌組推薦、單人精英任務、卡包建議) - 2024/12/16更新\n一樓\n最新\n本站所刊載之圖文內容等版權皆屬原廠商或原作者所有，非經同意請勿轉載\n巴 哈 姆 特 電 玩 資 訊 站 https://www.gamer.com.tw'), Document(metadata={'page': None, 'source': None}, page_content='【心得】PTCGP新手問題大合輯(牌組分享、活動牌組推薦、單人精英任務、卡包建議) - 2024/12/16更新 @寶可夢集換式卡牌遊戲口袋版 哈啦板 - 巴哈姆特\n找標題\n查詢標題相符的文章\n找作者\n查詢作者相符的文章\n找M文\n查詢被 MARK 的文章\n找全文\n搜尋本板文章、作者\n \n◄\n►\n1\n2\n3\n4\n...\n7\n【心得】PTCGP新手問題大合輯(牌組分享、活動牌組推薦、單人精英任務、卡包建議) - 2024/12/16更新\n綜合討論\n樓主\nJexSlam\n立志成為優文大師！\nJexSl

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

<class 'ragas.dataset_schema.EvaluationResult'> {'faithfulness': 1.0000}
<class 'float'> 1.0
  -DECISION: GENERATION IS GROUNDED IN DOCUMENTS-
---GRADE GENERATION vs QUESTION---
  -DECISION: GENERATION ADDRESSES QUESTION-


根據文件內容，提到的幾個隊伍中，噴火龍牌組被描述為擁有目前最強的攻擊力，適合喜歡大拳頭的玩家。而超夢牌組則具有高穩定度，運作起來比噴火龍順暢。皮卡丘牌組則以快攻為主，講求前中期快速擊敗對手。

因此，如果你想要快速獲勝，可以考慮使用噴火龍牌組或皮卡丘牌組。


In [58]:
run("在ptcg裡面有辦法第一回合進化出化石翼龍嗎?")

---ROUTE QUESTION---
  -ROUTETO VECTORSTORE-
---RETRIEVE---
[Document(metadata={'page': None, 'source': None}, page_content='JexSlam\nJexSlam\nBP\n-\n分享一下我的噴火龍牌組，\n希望可以幫助新手們找到一些方向\n基礎架構上，我是抄這篇，之後邊打邊修改～\n【心得】老噴牌組構築分享 (噴火龍討論串?!)\n以下分享我的改牌思路，歡迎提問和理性討論：\n1. 和上面網址的差別在於，我沒放蜥蜴，我是用\n六尾 X2+九尾 X2\n>> 原因是九尾有高達90的攻擊力，而且能量需求不大。平常小火龍除了進化外沒什麼用，但是有了九尾的90點傷害後，再加上小火龍的30點傷害，可以\n讓小火龍多一個尾刀的功能\n。\n2. 接著我發現，我的臉有夠黑，老是抽不到火焰鳥。\n>> \n火焰鳥是噴火龍牌組的核心\n，比噴火龍還重要。\n>> 沒有火焰鳥，就不能快速填能，等於噴火龍養不大。\n3. 這個遊戲的機制，就是\n起手一定有一隻基礎寶可夢\n。\n>> 也因此，我把六尾和九尾拔掉，換成紅牌和傷藥。\n>> 也就是\n我起手一定會有火焰鳥或小火龍\n。\n4. 起手一定有火焰鳥和小火龍的好處：\n>> 增加牌組穩定度（當然，後面的火恐龍和噴火龍還是要靠右手XD）\n>> 減少關鍵時刻被六尾九尾卡手的機會。\n目前我自己打下來，覺得穩定度和勝率有提升，\n當然，現在主流還是都有放除了噴火龍家族和火焰鳥以外的寶可夢，\n所以以上都是我自己實測改牌的經驗分享，\n歡迎大家多加比較和測試，也歡迎大家理性討論和分享你的牌組～\n3\n-\n回覆'), Document(metadata={'page': None, 'source': None}, page_content='JexSlam\nJexSlam\nBP\n-\n延續組牌的概念，我也組了一套超夢牌組，\n目前打了一週下來，勝率大約有8成，運作起來還算順暢。\n我組牌的核心概念，就是運用遊戲機制，\n「起手一定有一張基礎寶可夢」\n也因此我的超夢牌組，\n只有超夢EX，以及沙奈朵家族。\n>> 也就是我的\n起手一定會有一張超夢或拉魯拉絲\n我這套牌組，有個地方可以依照個人習慣調整：\n

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

<class 'ragas.dataset_schema.EvaluationResult'> {'faithfulness': 0.7500}
<class 'float'> 0.75
  -DECISION: GENERATION IS GROUNDED IN DOCUMENTS-
---GRADE GENERATION vs QUESTION---
  -DECISION: GENERATION ADDRESSES QUESTION-


根據文件內容，化石翼龍是透過「物品卡」秘密琥珀進化的，因為秘密琥珀是「物品卡」，所以不會占用開場必拿基礎卡的遊戲機制。因此，理論上在第一回合是有可能進化出化石翼龍的。


In [59]:
run("如何解決台灣高房價問題")

---ROUTE QUESTION---
  -ROUTE TO WEB SEARCH-
---WEB SEARCH---
[Document(metadata={}, page_content='a：解決高房價問題需政府、企業與社會各界共同努力，包括增加社會住宅興建、抑制炒作行為、提高薪資等。 我們如何找到房市的平衡點？ A：透過政府政策的調整、企業薪資的提升及社會共識的形成，才能找到房市的平衡點，實現合理居住的權利。'), Document(metadata={}, page_content='台灣房市的核心問題不在房價，高房價是「果」不是「因」，囤房稅、房地合一稅2.0都治不了果，重點要先找出「因」，才能對症下藥。 TOC制約因素法則(Theory Of Constraints)，分析出當前房市高房價的真正核心問題，並提出最終解決方案，才是房市治本的最佳政策。'), Document(metadata={}, page_content='當前的房價高不高？相信所有的人都會覺得很高，雖然身為業者，我也覺得很高，為什麼這麼高？是不是因為建商暴利？還是因為炒作？造成今日高房價的因素很多，絕對不是大家想的那麼單純，否則這個問題早就該被解決了，我要坦白的說，當初我也是因為覺得房價那麼高，建商「必定」享有高利'), Document(metadata={}, page_content='台灣高房價問題難解，日前有網友在《mobile01》發文表示，從台灣日治時期以來，以長線來看，不管是戰爭還是強震，房價始終上漲，從來都沒有跌'), Document(metadata={}, page_content='台灣的高房價問題使得許多人陷入租不起、買不下的困境。 無論是租房族的無殼蝸牛，還是買房族的屋奴人生，居住問題成為全民焦慮的來源。 本文深入剖析台灣居住困境的根本原因，探索各類政策對民眾的實際影響，並提供獨到的解決方案與未來展望，讓每')]


---CHECK DOCUMENT RELEVANCE TO QUESTION---
  -GRADE: DOCUMENT RELEVANT-
  -GRADE: DOCUMENT RELEVANT-
  -GRADE: DOCUMENT RELEVANT-
  -GRADE: DOCUMENT NOT RELEVANT-
  -GR

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

<class 'ragas.dataset_schema.EvaluationResult'> {'faithfulness': 1.0000}
<class 'float'> 1.0
  -DECISION: GENERATION IS GROUNDED IN DOCUMENTS-
---GRADE GENERATION vs QUESTION---
  -DECISION: GENERATION ADDRESSES QUESTION-


解決台灣高房價問題需政府、企業與社會各界共同努力，包括增加社會住宅興建、抑制炒作行為、提高薪資等。此外，透過政府政策的調整、企業薪資的提升及社會共識的形成，才能找到房市的平衡點，實現合理居住的權利。


In [60]:
run("中國高房價、經濟不景氣等問題，使用負債清零政策是否能提升經濟")

---ROUTE QUESTION---
  -ROUTE TO WEB SEARCH-
---WEB SEARCH---
[Document(metadata={}, page_content='不只清零急轉彎 中國房地產政策也在轉向 能挽救低迷的經濟嗎？ 2023年1月17日 ... 官媒新華社週五（1月13日）報導稱，中國央行已起草了21項工作任務來改善房地產企業的負債狀況，促進房地產市場平穩發展，這些措施將幫助房地產開發商的融資，包括調整對'), Document(metadata={}, page_content='2020年，根據中國人民銀行的數據，房地產直接投資超過一萬億美元。\n中國政府越來越擔心債務大山面臨崩塌的風險，決定進行干預。\n她說：「中國當局試圖通過要求房地產開發商放慢借貸速度來降低房價。為此，政府向銀行施壓，要求減少對開發商的借貸額。（干預）真正始於 2021 年初。」\n迅猛的急剎車，效果立竿見影。\n「系統內的流動性很快枯竭。今年，我們可以看到該法規的結果。對債務水平的過度監管是我們現在陷入困境的關鍵，」 王丹說。\n信貸收緊造成的一個引人注目的受害者是該國第二大房地產開發商。 恆大是一個巨大的房地產帝國，在中國數百個城市擁有數百萬套公寓。它還有超過3000億美元的財政義務，數百座未完工的住宅樓和許多憤怒之下封鎖了建築工地的供應商。\n而後果絕不僅限於恆大。\n房地產行業近三分之一的貸款現在被歸類為壞賬。\n王丹說：「我們看到，中國房地產開發商100強中大約60個已經違約。其餘40家公司即將面臨違約。因此，鑒於這種情況，所有的購房者都處於這種不安的狀態，許多人還在等待。已經支付了部分款項的許多人更希望確保自己的錢是安全的。」\n不僅僅是銀行面臨風險。\n對於中國的房地產投資者和潛在房主來說，這是一個新的、非常令人擔憂的威脅。\n她說：「對於中國消費者來說，這是全新的局面。人們總是認為經濟在上升，房地產市場永遠在上升。你什麼時候進去並不重要，房地產市場總是在上漲。但這一次，看起來最高領導層真的下了決心。我認為他們在說服人們房地產市場的最佳時期已經過去這一點上非常成功。」\n但中國房地產行業的債務只是影響中國前景的若干因素之一。\n圖像來源，Getty Images\n馬雲一度從公眾視線中消失，引起諸多揣測\n金融危機及其他\n世界銀行

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

<class 'ragas.dataset_schema.EvaluationResult'> {'faithfulness': 0.8462}
<class 'float'> 0.8461538461538461
  -DECISION: GENERATION IS GROUNDED IN DOCUMENTS-
---GRADE GENERATION vs QUESTION---
  -DECISION: GENERATION ADDRESSES QUESTION-


根據文件內容，中國的高房價和經濟不景氣問題與負債清零政策有關，但該政策是否能提升經濟仍然存在不確定性。文件提到，中國政府試圖通過要求房地產開發商放慢借貸速度來降低房價，並且這一政策的實施導致了系統內的流動性枯竭，進一步影響了經濟。

此外，房地產行業的債務問題和信貸收緊造成了許多開發商違約，這使得購房者和投資者感到不安，進而影響了消費信心。文件中提到，許多家庭的財富主要集中在房地產上，房地產市場的下滑使得家庭消費受到影響。

因此，雖然負債清零政策可能在某種程度上有助於解決債務問題，但其對經濟的提升效果並不明確，且可能需要配合其他措施來穩定市場信心和促進消費。


In [61]:
run("簡易分析台灣2024景氣狀況")

---ROUTE QUESTION---
  -ROUTE TO WEB SEARCH-
---WEB SEARCH---
[Document(metadata={}, page_content='gdp 年增率達6.56%。預計下半年國內景氣 穩定復甦，因此2024 全年的實質經濟成長率預測向上修正至3.88%。 隨著國內經濟景氣增溫，民眾對餐飲旅宿、休閒娛樂與交通運輸等消費需求 熱絡，第一季實質民間消費年增率為. 4.45%。零售業中汽機車業、綜合商品零售'), Document(metadata={}, page_content='預計下半年國內景氣穩定復甦，因此2024全年的實質經濟成長率預測向上修正至3.88%。 隨著國內經濟景氣增溫，民眾對餐飲旅宿、休閒娛樂與交通運輸等消費需求熱絡，第一季實質民間消費年增率為4.45%。'), Document(metadata={}, page_content='台灣2024年經濟現況 - 產業發展、總體市場分析 ... 台灣的經濟健康狀況經歷了明顯的下滑，相對於中國和美國等外部市場處於相對穩定的時期。 ... 變化、私人消費、零售狀況、地緣經濟、以及相應的國家貨幣與財政政策進行追蹤與分析；fr小組聚焦在各國與國際'), Document(metadata={}, page_content='2024年臺灣經濟情勢總展望─雪融春暖偶寒風 ... 展望2024年，雖然全球通膨趨緩，各國央行升息周期亦逐漸接近尾聲，全球景氣復甦仍略顯蹣跚。惟電子相關產品需求回溫，在高速運算與人工智慧等新興科技應用需求帶動下，我國外需與投資將可望恢復成長動能。'), Document(metadata={}, page_content='2024年第4季臺灣經濟預測 | CIER | 中華經濟研究院 About CIER The First Research Division The Third Research Division The Center for Economic Forecasting The Center for Financial and Economic Strategies The Taiwan WTO and RTA Center The Regional Development Study Center The C

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

<class 'ragas.dataset_schema.EvaluationResult'> {'faithfulness': 1.0000}
<class 'float'> 1.0
  -DECISION: GENERATION IS GROUNDED IN DOCUMENTS-
---GRADE GENERATION vs QUESTION---
  -DECISION: GENERATION ADDRESSES QUESTION-


根據文件內容，2024年台灣的經濟狀況預測顯示出穩定的復甦趨勢。以下是簡易分析：

1. **經濟成長率**：2024年的實質經濟成長率預測為3.88%至3.96%，顯示出經濟活動的增長。

2. **民間消費**：第一季的實質民間消費年增率為4.45%，顯示民眾對餐飲、旅宿、休閒娛樂及交通運輸等消費需求熱絡。

3. **內需與外貿**：內需和外貿雙引擎共同推動經濟成長，內需貢獻約3.07個百分點，民間消費貢獻1.50個百分點，顯示內需的強勁表現。

4. **通膨預測**：預計2024年CPI年增率約為2.17%，較前年的2.49%有所減少，顯示通膨壓力有所緩解。

5. **外部因素**：全球通膨趨緩及主要國際機構上修全球經濟成長率，對台灣的外需也有正面影響，尤其是AI等需求強勁的領域。

總體而言，2024年台灣的經濟展望是正面的，預計將在內需和外貿的共同推動下持續增長。
